In [205]:
# IMPORTANDO BIBLIOTECADAS

import pandas as pd
import pathlib
import win32com.client as win32

# IMPORTANDO BASES DE DADOS
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding = 'latin1', sep =';' )
emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')


# DEFINDINDO FUNÇÕES

def meta(valor, meta):   
    if valor < meta:
        cor  = 'red'
    else:
        cor = 'green'
    return cor

# CRIANDO TABELA PARA LOJA E DEFININDO INDICADORES
# CRIANDO TABELA PRA LOJA
vendas_completo = vendas.merge(lojas, on = 'ID Loja')

# CRIANDO DATA FRAME PARA CADA LOJA
dic_lojas = {}
for loja in lojas['Loja']:
    dic_lojas[loja] =  vendas_completo.loc[vendas_completo['Loja'] ==  loja, :]
     

# PUXANDO DIA INDICADOR
dia_indicador = vendas['Data'].max()

# CRIANDO PASTAS DE BACKUPS
caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')
arquivos_backup = caminho_backup.iterdir()
pastas = [arquivo.name for arquivo in arquivos_backup]
for loja in dic_lojas:
    if loja not in pastas:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()

# CRIANDO NOME DE CADA ARQUIVO 
    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo

# CRIANDO ARQUIVO EXCEL
    dic_lojas[loja].to_excel(local_arquivo)


# DEFININDO MEDAS
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdprodutos_dia = 4
meta_qtdprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

# BUSCANDO FATURAMENTO DE CADA  LOJA POR DIA E POR ANO
for loja in dic_lojas:
    vendas_loja = dic_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data'] == dia_indicador,:]

    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()

    # CALCULANDO DIVERDIDADE POR DIA E POR ANO

    qtd_produto_ano = len(vendas_loja['Produto'].unique())
    qtd_produto_dia = len(vendas_loja_dia['Produto'].unique())

    # CALCULANDO TICKET MEDIO POR DIA E POR ANO

    ticket_medio_ano = faturamento_ano / len(vendas_loja['Código Venda'].unique())
    ticket_medio_dia =  faturamento_dia / len(vendas_loja_dia['Código Venda'].unique())

    # CRIANDO EMAIL COM INDICADORES PARA GERETNE DE CADA LOJA
    outlook = win32.Dispatch('outlook.application')
    nome_gerenta = emails.loc[emails['Loja'] == loja , 'Gerente'].values[0]
    mail = outlook.CreateItem(0)
    mail.To = emails.loc[emails['Loja'] == loja , 'E-mail'].values[0]
    mail.Subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'

    cor_fat_dia = meta(faturamento_dia, meta_faturamento_dia)
    cor_fat_ano = meta(faturamento_ano, meta_faturamento_ano)

    cor_qtd_dia = meta(qtd_produto_dia, meta_qtdprodutos_dia)
    cor_qtd_ano = meta(qtd_produto_ano, meta_qtdprodutos_ano)

    cor_ticket_dia = meta(ticket_medio_dia, meta_ticketmedio_dia)
    cor_ticket_ano = meta(ticket_medio_ano, meta_ticketmedio_ano)

    mail.HTMLBody = f'''

    <p> Bom dia, <strong> {nome_gerenta} </strong> </p>

    <p> O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month}) </strong> da loja <strong> {loja} </strong> foi: </p>

    <h2> Relatório do dia </h2>

    <table>
    <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenario Dia</th>
    </tr>
    <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_dia:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
        <td style="text-align: center"><font color = {cor_fat_dia}>■</font></td>
    </tr>
    <tr>
        <td>Diversidade de produtos</td>
        <td style="text-align: center">{qtd_produto_dia}</td>
        <td style="text-align: center">{meta_qtdprodutos_dia}</td>
        <td style="text-align: center"><font color = {cor_qtd_dia}>■</font></td>
    </tr>
    <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_dia:.2f}</td>
        <td style="text-align: center"><font color = {cor_ticket_dia}>■</font></td>
    </tr>
    </table>
    <br>

    <h2>Relatório do Ano</h2>

    <table>
    <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Dia</th>
        <th>Cenario Dia</th>
    </tr>
    <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_ano:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
        <td style="text-align: center"><font color = {cor_fat_ano}>■</font></td>
    </tr>
    <tr>
        <td>Diversidade de produtos</td>
        <td style="text-align: center">{qtd_produto_ano}</td>
        <td style="text-align: center">{meta_qtdprodutos_ano}</td>
        <td style="text-align: center"><font color = {cor_qtd_ano}>■</font></td>
    </tr>
    <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_ano:.2f}</td>
        <td style="text-align: center"><font color = {cor_ticket_ano}>■</font></td>
    </tr>
    </table>

    </body>
    </html>


    <p> Segue em anexo a planilha com todos os dados para mais detalhes. </p>

    <p> <strong> Qualquer dúvida estou à disposição </strong> </p>
    <p> <strong> Att, Geovane Firmino </strong></p>

    '''
    # ANEXANDO ARQUIVOS
    attachment = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    mail.Attachments.Add(str(attachment))
    mail.Send()
    

# CRIANDO RANKINGS

fat_loja_ano = vendas_completo.groupby('Loja')[['Valor Final']].sum().sort_values(by = 'Valor Final', ascending= False)
nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.month, dia_indicador.day)
fat_loja_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))


# FATURAMENTO POR DIA
vendas_dia = vendas_completo.loc[vendas_completo['Data'] == dia_indicador, :]
fat_loja_dia = vendas_dia.groupby('Loja')[['Valor Final']].sum().sort_values(by = 'Valor Final', ascending= False)

# SALAVDNO EXCEL
nome_arquivo = '{}_{}_Ranking Dia.xlsx'.format(dia_indicador.month, dia_indicador.day)
fat_loja_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))


# CRIANDO E-MAIL PARA SER ENVIADO PARA DIRETORES COM OS ARQUIVOS DE RANKING
outlook = win32.Dispatch('outlook.application')
nome_gerenta = emails.loc[emails['Loja'] == 'Diretoria' , 'E-mail'].values[0]
mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja'] == loja , 'E-mail'].values[0]
mail.Subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'
mail.Body = f'''
Prezados, bom dia.

Melhor loja do dia em faturamento: Loja {fat_loja_dia.index[0]} com faturamento R$ {fat_loja_dia.iloc[0, 0]:.2f}
Pior loja do dia em faturamento: Loja {fat_loja_dia.index[-1]} com faturamento R$ {fat_loja_dia.iloc[-1,-1]:.2f}

Melhor loja do dia em faturamento: Loja {fat_loja_ano.index[0]} com faturamento R$ {fat_loja_ano.iloc[0,0]:.2f}
Pior loja do dia em faturamento: Loja {fat_loja_ano.index[-1]} com faturamento R$ {fat_loja_ano.iloc[-1,-1]:.2f}

Segue em anexo dos rankings do ano e dia de todas as lojas.

Qualquer dúvida estou a disposição.

Att,
Geovane Firmino
'''

# ANEXANDO ARQUIVOS
attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx'
mail.Attachments.Add(str(attachment))
attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Dia.xlsx'
mail.Attachments.Add(str(attachment))
mail.Send()

# MOSTRANDO QUE TODO PROCESSO FOI CONCLUIDO

print('Processo de envios concluido!')

C:\Users\Firmino\AppData\Local\Temp\ipykernel_23612\1907417165.py:76: RuntimeWarning: invalid value encountered in scalar divide
  ticket_medio_dia =  faturamento_dia / len(vendas_loja_dia['Código Venda'].unique())


Processo de envios concluido!
